# ТОП 20 продаж SKU в категории 

In [ ]:
import pandas as pd

# Читаем Excel-файл
df = pd.read_excel('Top_20_in_group.xlsx')

# Переименуем столбцы для удобства (если названия отличаются, подставьте свои)
df.columns = ['Классификатор_ур1', 'Классификатор_ур2', 'Код_номенклатуры', 'Товар', 'Сумма_продаж']

# Группируем данные по "Классификатор_ур1" и сортируем внутри каждой группы по продажам
# Также берем топ-20 уникальных товаров (по "Код_номенклатуры" или "Товар", так как они уникальны)
top_20_by_group = (df.groupby('Классификатор_ур1')
                   .apply(lambda x: x.sort_values(by='Сумма_продаж', ascending=False)
                   .drop_duplicates(subset=['Код_номенклатуры'])  # Убираем дубликаты по уникальному коду
                   .head(20))  # Берем топ-20
                   .reset_index(drop=True))

# Сохраняем результат в новый Excel-файл
top_20_by_group.to_excel('Top_20_by_Classifier1.xlsx', index=False)

print("Топ-20 товаров по продажам для каждой категории сохранены в 'Top_20_by_Classifier1.xlsx'")

# Сглаживание Пиков

In [ ]:
import pandas as pd          # простоге скользящее среднее (SMA, Simple Moving Average)
import matplotlib.pyplot as plt

# === 1. Загрузка данных ===
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# === 2. Сглаживание по каждому SKU ===
df = df.sort_values(['Код товара', 'Дата'])  # Сортировка по SKU и дате
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

# === 3. График для конкретного SKU ===
specific_sku = '187909'  # Указываем код как строку
# Пробуем преобразовать коды в строковый тип для единообразия
df['Код товара'] = df['Код товара'].astype(str).str.strip()

sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    print(f"Найдены данные для SKU {specific_sku}: {len(sku_df)} строк")
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (SMA 7)', linewidth=2)
    plt.title(f'Сглаживание продаж для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден в данных после преобразования.")

# === 4. Сохранение в Excel ===
# output_path = 'SKU_smooth_output.xlsx'
# df.to_excel(output_path, index=False)
# print(f"Файл сохранён как: {output_path}")

In [ ]:
import pandas as pd        #  Скользящее медианное сглаживание
import matplotlib.pyplot as plt

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Скользящая медиана
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.rolling(window=7, min_periods=1).median()
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (Медиана 7)', linewidth=2)
    plt.title(f'Сглаживание медианой для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

In [ ]:
import pandas as pd            #Экспоненциальное сглаживание (EMA)
import matplotlib.pyplot as plt

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Экспоненциальное сглаживание
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: x.ewm(span=7, adjust=False).mean()
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (EMA 7)', linewidth=2)
    plt.title(f'Экспоненциальное сглаживание для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

In [ ]:
import pandas as pd                 #Savitzky-Golay
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Загрузка данных
file_path = 'SKU for prophet.xlsx'
df = pd.read_excel(file_path, sheet_name='Data')

# Сортировка
df = df.sort_values(['Код товара', 'Дата'])
df['Код товара'] = df['Код товара'].astype(str).str.strip()

# Применение Savitzky-Golay фильтра
df['Продажи_сглажено'] = df.groupby('Код товара')['Продажи, кг'].transform(
    lambda x: savgol_filter(x, window_length=7, polyorder=2) if len(x) >= 7 else x
)

# График для конкретного SKU
specific_sku = '187909'
sku_df = df[df['Код товара'] == specific_sku]

if not sku_df.empty:
    plt.figure(figsize=(16, 8))
    plt.plot(sku_df['Дата'], sku_df['Продажи, кг'], label='Оригинальные данные', alpha=0.5)
    plt.plot(sku_df['Дата'], sku_df['Продажи_сглажено'], label='Сглаженные (Savitzky-Golay)', linewidth=2)
    plt.title(f'Сглаживание Savitzky-Golay для SKU {specific_sku}')
    plt.xlabel('Дата')
    plt.ylabel('Продажи, кг')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"Код товара {specific_sku} не найден.")

# Схлопывает магазины и строки в одну


In [ ]:
import pandas as pd

# Чтение данных из Excel-файла
df = pd.read_excel('1.xlsx')

# Группируем по магазину, собираем виды деятельности в список и суммируем доход
result = df.groupby('Магазин').agg({
    'Вид деятельности субарендатора': lambda x: ', '.join(x),
    'Сумма арендной платы, руб': 'sum'
}).reset_index()

# Формируем итоговую строку для каждого магазина
result['Итог'] = 'Субаренда: ' + result['Вид деятельности субарендатора'] + '. Общий доход от СА – ' + result['Сумма арендной платы, руб'].astype(str) + ' р.'

# Удаляем промежуточные столбцы
result = result[['Магазин', 'Итог']]

# Сохраняем результат в новый Excel-файл
result.to_excel('result_table.xlsx', index=False)

# Выводим результат для проверки
print(result)

# Быстрый Исследовательский анализ(EDA) и визуализация

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Загрузка данных
df = pd.read_excel('test ydata.xlsx')

# Создание отчёта
profile = ProfileReport(df, title="Excel Data Report", explorative=True)

# Сохранение отчёта в HTML
profile.to_file("report.html")

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from datetime import datetime

# Загрузка данных (замените на путь к вашему файлу)
# Пример: df = pd.read_excel('your_data.xlsx')
# Для демонстрации используем ваши примерные данные
data = {
    'Дата': ['01.04.2025'] * 11,
    'Классификатор ур.1': ['Молочная гастрономия'] * 11,
    'Классификатор ур.2': [
        'Детская молочная продукция', 'Майонез', 'Масложировые продукты', 'Молоко',
        'Молоко д.х, сливки, напитки раст.', 'Молочные консервы', 'Соврем.кислом.прод-ия',
        'Сырная витрина', 'Сыры', 'Творог, творожные изделия', 'Традиц.кислом.прод-ия'
    ],
    'Остатки, кг': [
        7348.75, 35610.95385, 8976.99746, 34066.2508, 53377.595, 21814.4,
        25937.384, 3342.865, 36176.84494, 9430.88348, 40741.0521
    ]
}
df = pd.DataFrame(data)

# Преобразование столбца 'Дата' в datetime
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

# Определение числовых столбцов (метрик) и категориальных столбцов
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_columns = ['Классификатор ур.1', 'Классификатор ур.2']

# Инициализация Dash приложения
app = Dash(__name__)

# Макет дашборда
app.layout = html.Div([
    html.H1("Динамический дашборд для анализа данных", style={'textAlign': 'center'}),
    
    # Фильтры
    html.Div([
        html.Label("Выберите метрику:"),
        dcc.Dropdown(
            id='metric-dropdown',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            value=numeric_columns[0],  # Метрика по умолчанию
            style={'width': '50%'}
        ),
    ], style={'margin': '10px'}),
    
    html.Div([
        html.Label("Выберите категорию:"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{'label': col, 'value': col} for col in categorical_columns],
            value=categorical_columns[-1],  # Категория по умолчанию
            style={'width': '50%'}
        ),
    ], style={'margin': '10px'}),
    
    html.Div([
        html.Label("Выберите диапазон дат:"),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed=df['Дата'].min(),
            max_date_allowed=df['Дата'].max(),
            initial_visible_month=df['Дата'].max(),
            start_date=df['Дата'].min(),
            end_date=df['Дата'].max()
        ),
    ], style={'margin': '10px'}),
    
    # Графики
    html.Div([
        dcc.Graph(id='pie-chart'),
        dcc.Graph(id='bar-chart')
    ])
])

# Callback для обновления графиков
@app.callback(
    [Output('pie-chart', 'figure'), Output('bar-chart', 'figure')],
    [Input('metric-dropdown', 'value'),
     Input('category-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_charts(selected_metric, selected_category, start_date, end_date):
    # Фильтрация данных по датам
    filtered_df = df[(df['Дата'] >= start_date) & (df['Дата'] <= end_date)]
    
    # Группировка данных для графиков
    grouped_df = filtered_df.groupby(selected_category)[selected_metric].sum().reset_index()
    
    # Круговая диаграмма
    pie_fig = px.pie(
        grouped_df,
        values=selected_metric,
        names=selected_category,
        title=f'Распределение {selected_metric} по {selected_category}',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    
    # Гистограмма
    bar_fig = px.bar(
        grouped_df,
        x=selected_category,
        y=selected_metric,
        title=f'{selected_metric} по {selected_category}',
        color=selected_category,
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    bar_fig.update_layout(xaxis_title=selected_category, yaxis_title=selected_metric)
    
    return pie_fig, bar_fig

# Запуск приложения
if __name__ == '__main__':
    app.run_server(debug=True)

# Автоматизация. Открытие.Обновление.Отправка

In [ ]:
import win32com.client
import os
import schedule
import time

# 1. Обновление сводных таблиц в Excel-файле
def update_pivot_tables(file_path):
    try:
        # Проверка существования файла
        if not os.path.exists(file_path):
            print(f"Файл {file_path} не найден.")
            return False
        
        # Инициализация Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False         # Скрыть Excel
        excel.DisplayAlerts = False  # Отключить предупреждения
        
        # Открытие файла
        workbook = excel.Workbooks.Open(file_path)
        
        # Обновление всех сводных таблиц на всех листах
        for worksheet in workbook.Worksheets:
            for pivot_table in worksheet.PivotTables():
                pivot_table.RefreshTable()
        
        # Сохранение и закрытие файла
        workbook.Save()
        workbook.Close()
        excel.Quit()
        
        print(f"Excel-файл {file_path} успешно обновлен (сводные таблицы).")
        return True
    except Exception as e:
        print(f"Ошибка при обновлении сводных таблиц: {e}")
        return False
    finally:
        # Очистка объектов
        workbook = None
        excel = None

# 2. Отправка email через Outlook
def send_email_outlook(recipients, cc, subject, body, attachment_path):
    try:
        # Инициализация Outlook
        outlook = win32com.client.Dispatch("Outlook.Application")
        
        # Создание нового письма
        mail = outlook.CreateItem(0)  # 0 = olMailItem
        
        # Настройка параметров письма
        mail.To = recipients
        mail.CC = cc
        mail.Subject = subject
        mail.Body = body
        
        # Добавление вложения
        if os.path.exists(attachment_path):
            mail.Attachments.Add(attachment_path)
        else:
            print(f"Вложение {attachment_path} не найдено.")
            return False
        
        # Отправка письма
        mail.Send()
        print("Письмо успешно отправлено через Outlook!")
        return True
    except Exception as e:
        print(f"Ошибка при отправке письма: {e}")
        return False
    finally:
        # Очистка объектов
        mail = None
        outlook = None

# 3. Основная задача
def job():
    # Параметры
    excel_file = r"C:/Users/bondarenkovv/Desktop/Python/алгоритмы ексель/test ydata.xlsx"
    recipients = "bondarenkovv@pudmarket.ru"
    cc = "bondarenkovv@pudmarket.ru"
    subject = "Отчет"
    body = "Добрый день! Отчёт во вложении."
    
    # Обновление сводных таблиц и отправка письма
    if update_pivot_tables(excel_file):
        send_email_outlook(recipients, cc, subject, body, excel_file)

# 4. Планирование задачи
schedule.every().day.at("09:00").do(job)

# 5. Запуск планировщика
def main():
    print("Планировщик запущен. Ожидание выполнения задачи в 09:00...")
    while True:
        schedule.run_pending()
        time.sleep(60)  # Проверка каждую минуту

if __name__ == '__main__':
    main()

# Подключение к серверу

In [ ]:
import pyodbc
import pandas as pd

# Параметры подключения
server = 'Avtozakazdb.corp.prodshops.ru'
database = 'Avtozakaz'
table = 'dbo.SalesDayAggregate'

# Строка подключения для Windows-аутентификации
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes"
)

# Запрос для получения данных за последний день
query = """
SELECT *
FROM dbo.SalesDayAggregate
WHERE Date = (SELECT MAX(Date) FROM dbo.SalesDayAggregate);
"""
try:
    # Подключение к базе данных
    print(f"Подключение к базе данных {database} с Windows-аутентификацией...")
    connection = pyodbc.connect(connection_string)

    # Используем pandas для выполнения запроса и загрузки данных в DataFrame
    print("Извлечение данных...")
    df = pd.read_sql(query, connection)

    # Проверяем, что данные получены
    if df.empty:
        print("Данные не найдены. Проверьте столбец SaleDate или наличие данных.")
    else:
        print(f"Получено {len(df)} строк за последний день.")

        # Сохранение данных в Excel
        excel_file = "SalesDayAggregate_LastDay.xlsx"
        df.to_excel(excel_file, index=False, engine='openpyxl')
        print(f"Данные сохранены в файл: {excel_file}")

except pyodbc.Error as ex:
    print("Ошибка при подключении к базе данных или выполнении запроса:", ex)
    print("Проверьте настройки сервера или обратитесь к администратору.")

finally:
    if 'connection' in locals():
        connection.close()
        print("Соединение закрыто.")

# Переворот таблицы

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Читаем файл
df = pd.read_excel('переворот таблицы.xlsx', dtype=str)

# Находим столбцы-недели (все числовые, кроме фиксированных)
fixed = ['ПУД', 'АК', 'Наименование']
week_cols = [(c, int(c)) for c in df.columns if c not in fixed and str(c).strip().isdigit()]

# Понедельник недели 1 в 2026 = 29 декабря 2025
def get_monday(week):  
    return datetime(2025, 12, 29) + timedelta(weeks=week-1)

rows = []
for _, row in df.iterrows():
    naim = row.get('Наименование', '')
    ak = row.get('АК', '')

    for col, week in week_cols:
        val = row[col]
        if pd.isna(val) or not str(val).strip(): 
            continue
        kg = float(str(val).replace(',', '.'))

        monday = get_monday(week)
        days = [(monday + timedelta(days=i)).month for i in range(7)]

        # Сколько дней в каждом месяце
        for month in set(days):
            days_in_month = days.count(month)
            kg_month = kg / 7 * days_in_month
            rows.append({
                'Месяц': month,
                'Неделя': week,
                'Наименование': naim,
                'АК': ak,
                'кг': round(kg_month, 3)
            })

# Результат
result = pd.DataFrame(rows)
result = result.sort_values(['Месяц', 'Неделя', 'Наименование']).reset_index(drop=True)
result.to_excel('Транспонирование_таблицы.xlsx', index=False)

print("Готово! Файл: Транспонирование_таблицы.xlsx")

Готово! Файл: результат.xlsx


# Дефицит Излишки товара

In [ ]:
import pandas as pd

def analyze_stock_corrected(file_path, min_stock=5, shortage_factor=0.5, excess_factor=2.0, max_shortage_threshold=30, min_sales_qty=5):
    # Чтение данных из Excel
    df = pd.read_excel(file_path)
    
    # Проверка наличия необходимых столбцов
    required_columns = ['Магазин', 'Код номенклатуры', 'Номенклатура', 'Кол-во продано, шт.', 
                        'Остатки, шт.', 'Оборачиваемость, дни', 'Запас в днях', 'Торговых дней']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("В файле отсутствуют необходимые столбцы")
    
    # Фильтрация активных товаров
    df_active = df[df['Торговых дней'] > 0].copy()
    
    # Определение статуса
    def determine_status(row):
        stock_days = row['Запас в днях']
        turnover_days = row['Оборачиваемость, дни']
        stock_qty = row['Остатки, шт.']
        sales_qty = row['Кол-во продано, шт.']
        
        # Динамические пороги с ограничением
        shortage_threshold = min(shortage_factor * turnover_days, max_shortage_threshold)
        excess_threshold = excess_factor * turnover_days
        
        # Условие дефицита: низкий запас в днях ИЛИ низкие остатки, ПЛЮС достаточные продажи
        if (stock_days < shortage_threshold or stock_qty < min_stock) and sales_qty >= min_sales_qty:
            return 'Дефицит'
        # Условие избытка: высокий запас в днях И низкие продажи
        elif stock_days > excess_threshold and sales_qty < stock_qty:
            return 'Избыток'
        else:
            return 'Норма'
    
    df_active['Статус'] = df_active.apply(determine_status, axis=1)
    
    # Разделение на дефицит и избыток
    shortage = df_active[df_active['Статус'] == 'Дефицит'][['Магазин', 'Код номенклатуры', 'Номенклатура', 
                                                           'Кол-во продано, шт.', 'Остатки, шт.', 
                                                           'Оборачиваемость, дни', 'Запас в днях']]
    excess = df_active[df_active['Статус'] == 'Избыток'][['Магазин', 'Код номенклатуры', 'Номенклатура', 
                                                         'Кол-во продано, шт.', 'Остатки, шт.', 
                                                         'Оборачиваемость, дни', 'Запас в днях']]
    
    # Сохранение результатов
    with pd.ExcelWriter('stock_analysis_result_corrected.xlsx') as writer:
        shortage.to_excel(writer, sheet_name='Дефицит', index=False)
        excess.to_excel(writer, sheet_name='Избыток', index=False)
    
    # Вывод статистики
    print(f"Найдено {len(shortage)} позиций с дефицитом и {len(excess)} позиций с избытком.")
    print("Результаты сохранены в 'stock_analysis_result_corrected.xlsx'")

# Пример использования
if __name__ == "__main__":
    file_path = 'табак.xlsx'
    analyze_stock_corrected(file_path)

In [ ]:
import pandas as pd   # Для всего ассортимента + Срок Годности

def analyze_stock_with_storage(file_path, min_stock=5, shortage_factor=0.5, excess_factor=2.0, 
                              max_shortage_threshold=30, min_sales_qty=5, storage_factor=0.2):
    # Чтение данных из Excel
    df = pd.read_excel(file_path)
    
    # Проверка наличия необходимых столбцов
    required_columns = ['Магазин', 'Код номенклатуры', 'Номенклатура', 'Срок хранения', 
                        'Кол-во продано, шт.', 'Остатки, шт.', 'Оборачиваемость, дни', 
                        'Запас в днях', 'Торговых дней']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("В файле отсутствуют необходимые столбцы")
    
    # Фильтрация активных товаров
    df_active = df[df['Торговых дней'] > 0].copy()
    
    # Определение статуса
    def determine_status(row):
        stock_days = row['Запас в днях']
        turnover_days = row['Оборачиваемость, дни']
        stock_qty = row['Остатки, шт.']
        sales_qty = row['Кол-во продано, шт.']
        storage_days = row['Срок хранения']
        
        # Динамические пороги с учетом срока хранения
        shortage_threshold = min(shortage_factor * turnover_days, storage_factor * storage_days, max_shortage_threshold)
        excess_threshold = min(excess_factor * turnover_days, 0.8 * storage_days)
        
        # Условие дефицита: низкий запас в днях ИЛИ низкие остатки, ПЛЮС достаточные продажи
        if (stock_days < shortage_threshold or stock_qty < min_stock) and sales_qty >= min_sales_qty:
            return 'Дефицит'
        # Условие избытка: высокий запас в днях И низкие продажи
        elif stock_days > excess_threshold and sales_qty < stock_qty:
            return 'Избыток'
        else:
            return 'Норма'
    
    df_active['Статус'] = df_active.apply(determine_status, axis=1)
    
    # Разделение на дефицит и избыток, включая 'Торговых дней'
    shortage = df_active[df_active['Статус'] == 'Дефицит'][['Магазин', 'Код номенклатуры', 'Номенклатура', 
                                                           'Срок хранения', 'Кол-во продано, шт.', 
                                                           'Остатки, шт.', 'Оборачиваемость, дни', 
                                                           'Запас в днях', 'Торговых дней']]
    excess = df_active[df_active['Статус'] == 'Избыток'][['Магазин', 'Код номенклатуры', 'Номенклатура', 
                                                         'Срок хранения', 'Кол-во продано, шт.', 
                                                         'Остатки, шт.', 'Оборачиваемость, дни', 
                                                         'Запас в днях', 'Торговых дней']]
    
    # Сохранение результатов
    with pd.ExcelWriter('stock_analysis_result_with_storage.xlsx') as writer:
        shortage.to_excel(writer, sheet_name='Дефицит', index=False)
        excess.to_excel(writer, sheet_name='Избыток', index=False)
    
    # Вывод статистики
    print(f"Найдено {len(shortage)} позиций с дефицитом и {len(excess)} позиций с избытком.")
    print("Результаты сохранены в 'stock_analysis_result_with_storage.xlsx'")

# Пример использования
if __name__ == "__main__":
    file_path = 'табак.xlsx'
    analyze_stock_with_storage(file_path)